In [6]:
import cv2
import numpy as np
import time
import mediapipe as mp
from sklearn.cluster import KMeans

# Initialize MediaPipe hand detection
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

def create_background(cap, num_frames=30):
    print("Capturing background. Please move out of frame.")
    backgrounds = []
    for i in range(num_frames):
        ret, frame = cap.read()
        if ret:
            backgrounds.append(frame)
        else:
            print(f"Warning: Could not read frame {i+1}/{num_frames}")
        time.sleep(0.1)
    if backgrounds:
        return np.median(backgrounds, axis=0).astype(np.uint8)
    else:
        raise ValueError("Could not capture any frames for background")

def create_mask(frame, lower_color, upper_color):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8), iterations=1)
    return mask

def apply_kmeans_color_quantization(frame, k=16):
    data = frame.reshape((-1, 3))
    kmeans = KMeans(n_clusters=k, random_state=0)
    labels = kmeans.fit_predict(data)
    quantized_data = kmeans.cluster_centers_.astype(np.uint8)[labels]
    return quantized_data.reshape(frame.shape)

def apply_cloak_effect_with_video(frame, mask, galaxy_video):
    ret, galaxy_frame = galaxy_video.read()
    if not ret:
        galaxy_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, galaxy_frame = galaxy_video.read()
    galaxy_resized = cv2.resize(galaxy_frame, (frame.shape[1], frame.shape[0]))
    mask_inv = cv2.bitwise_not(mask)
    fg = cv2.bitwise_and(frame, frame, mask=mask_inv)
    bg = cv2.bitwise_and(galaxy_resized, galaxy_resized, mask=mask)
    return cv2.add(fg, bg)

def detect_gesture(frame):
    # Flip the frame horizontally for a later mirror view
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:
            # Draw landmarks
            mp_draw.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Gesture detection: Example - Checking for a "thumbs-up" gesture
            thumb_tip = landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            
            # You can improve this logic by checking angles between the thumb and index finger
            if thumb_tip.x < index_tip.x:
                return 'toggle_cloak'
            else:
                return 'no_gesture'
    return 'no_gesture'

def main():
    print("OpenCV version:", cv2.__version__)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    galaxy_video = cv2.VideoCapture('galaxy.mp4')
    if not galaxy_video.isOpened():
        print("Error: Could not open galaxy video.")
        cap.release()
        return

    try:
        background = create_background(cap)
    except ValueError as e:
        print(f"Error: {e}")
        cap.release()
        galaxy_video.release()
        return

    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])

    print("Starting main loop. Press 'q' to quit.")
    cloak_enabled = True
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            time.sleep(1)
            continue

        # Detect gesture
        gesture = detect_gesture(frame)
        
        # Toggle cloak effect based on gesture
        if gesture == 'toggle_cloak':
            cloak_enabled = not cloak_enabled
            print("Cloak toggled:", "Enabled" if cloak_enabled else "Disabled")
        
        # Apply K-means color quantization
        quantized_frame = apply_kmeans_color_quantization(frame, k=16)
        mask = create_mask(quantized_frame, lower_blue, upper_blue)

        # Only apply the cloak effect if it's enabled
        if cloak_enabled:
            result = apply_cloak_effect_with_video(quantized_frame, mask, galaxy_video)
        else:
            result = quantized_frame  # Show the normal frame if cloak is disabled

        cv2.imshow('Augmented Invisibility Cloak', result)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    galaxy_video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'mediapipe'

In [4]:
!pip install --user mediapipe


  Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached jax-0.4.38-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.38-cp312-cp312-win_amd64.whl.metadata (1.1 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl (51.0 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached jax-0.4.38-py3-none-any.whl (2.2 MB)
Using cached jaxlib-0.4.38-cp312-cp312-win_amd64.whl (64.3 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
